In [1]:
import numpy as np
import pandas as pd
import requests
import json

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data1 = pd.read_csv('train1.csv')
data1[:10]

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,Germany,Mineral industry,Installations for the production of cement cli...,https://registry.gdi-de.org/id/de.ni.mu/062217...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,AIR,Carbon dioxide (CO2),2015,10,...,EUROPE,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,2,Mr. Jacob Ortega,7cdb5e74adcb2ffaa21c1b61395a984f
1,Italy,Mineral industry,Installations for the production of cement cli...,IT.CAED/240602021.FACILITY,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,AIR,Nitrogen oxides (NOX),2018,9,...,EUROPE,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,1,Ashlee Serrano,cd1dbabbdba230b828c657a9b19a8963
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,ES.CAED/001966000.FACILITY,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,AIR,Methane (CH4),2019,2,...,EUROPE,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,2,Vincent Kemp,5011e3fa1436d15b34f1287f312fbada
3,Czechia,Energy sector,Thermal power stations and other combustion in...,CZ.MZP.U422/CZ34736841.FACILITY,Elektrárny Prunéřov,Kadaň,AIR,Nitrogen oxides (NOX),2012,8,...,EUROPE,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,Carol Gray,37a6d7a71c4f7c2469e4f01b70dd90c2
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,http://paikkatiedot.fi/so/1002031/pf/Productio...,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,AIR,Methane (CH4),2018,12,...,EUROPE,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,2,Blake Ford,471fe554e1c62d1b01cc8e4e5076c61a
5,Switzerland,Energy sector,Mineral oil and gas refineries,CH.CAED/000000011.Facility,Varo Refining Cressier SA / Raffinerie de Cres...,Cressier,AIR,Nitrogen oxides (NOX),2009,11,...,EUROPE,13.610384,16.054021,18.476185,0.218463,1.695830,3.081757,2,Jonathan Evans,9ecac1661f9a6d2ea27ea6582db34d9f
6,France,Mineral industry,"Installations for the manufacture of glass, in...",FR.CAED/11626.FACILITY,VERALLIA,COGNAC,AIR,Carbon dioxide (CO2),2008,5,...,EUROPE,12.816569,15.940397,21.873807,10.954453,13.806014,16.682482,1,Kara Martin,1eb1fba9d2767e70c428514f7299acc0
7,Poland,Paper and wood production and processing,Industrial plants for the production of paper ...,PL.MŚ/000000138.FACILITY,Arctic Paper Kostrzyn S.A.,Kostrzyn nad Odrą,AIR,Carbon dioxide (CO2),2011,4,...,EUROPE,9.143964,14.174349,19.879915,11.915887,12.930775,17.699905,2,David Nichols,90ada31eb6075ca41d9e7b23d27b1526
8,United Kingdom,Energy sector,Thermal power stations and other combustion in...,UK.CAED/BEISOffsh-Bleo-Holm.FACILITY,Bleo Holm FPSO,--,AIR,Carbon dioxide (CO2),2010,6,...,EUROPE,20.766119,21.205965,26.255209,6.348797,7.877442,11.130713,17,Frederick Chapman,cfab1ba8c67c7c838db98d666f02a132
9,France,Chemical industry,Chemical installations for the production on a...,FR.CAED/3839.FACILITY,USINE DE GONFREVILLE,GONFREVILLE-L'ORCHER,AIR,Carbon dioxide (CO2),2014,11,...,EUROPE,17.949222,20.947898,27.992034,9.633089,10.736422,12.305046,0,Sheena Conner,bf61dcbfc9487dc9dd63e8100d0b057e


In [3]:
#He añadido más argunmentos a la función csv ya que era necesario para poder interpretarlo
data2 = pd.read_csv('train2.csv', sep = ';',on_bad_lines='skip')
data2[:10]

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,Germany,Waste and wastewater management,Installations for the incineration of non-haza...,https://registry.gdi-de.org/id/de.hh/pf.bube-e...,MVR Müllverwertung Rugenberger Damm GmbH & Co. KG,Hamburg,AIR,Nitrogen oxides (NOX),2012,4,...,EUROPE,13.006440,17.328013,22.819874,13.642167,13.524782,15.210716,0,Teresa Martin,35d7df6ed3d93be2927d14acc5f1fc9a
1,France,Energy sector,Thermal power stations and other combustion in...,FR.EEA/6288.FACILITY,SOCIETE DE COGENERATION,TAVAUX,AIR,Nitrogen oxides (NOX),2007,3,...,EUROPE,12.601338,16.415961,20.870744,12.425496,11.640683,14.170232,1,Teresa Monroe,8079579bf1d5379ea893be33dbb997d5
2,France,Energy sector,Thermal power stations and other combustion in...,FR.CAED/12066.FACILITY,CPCU ST-OUEN III,SAINT-OUEN,AIR,Carbon dioxide (CO2),2008,11,...,EUROPE,17.051488,18.558361,22.729832,10.676109,12.530537,14.036677,1,Brian Johnson,38fde98415bd374755bb341af3241c4f
3,Germany,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,https://registry.gdi-de.org/id/de.nw.inspire.p...,Deponie Haus Forst REMONDIS GmbH Rheinland,Kerpen,AIR,Methane (CH4),2009,2,...,EUROPE,9.345776,14.584978,22.153539,1.158088,1.424305,4.768707,1,David Jackson,8b73a54f4cb8ff07dd3e956bfa42b196
4,Estonia,Energy sector,Installations for gasification and liquefaction,EE.KAUR.TTR/76.FACILITY,"Enefit Energiatootmine AS, Auvere põlevkiviõli...","Auvere küla, Narva-Jõesuu linn",AIR,Carbon dioxide (CO2),2016,7,...,EUROPE,17.122838,18.382589,20.621925,8.620337,8.336314,12.852514,0,Holly Graves,cffe5169a23e2951963dc5e5da3fcd97
5,Sweden,Energy sector,Thermal power stations and other combustion in...,SE.CAED/10020920.Facility,Karlskoga kraftvärmeverk,KARLSKOGA,AIR,Carbon dioxide (CO2),2019,10,...,EUROPE,17.063911,20.519818,23.084057,11.375806,12.972962,16.259667,2,Karen Torres,8f2e9927f13253155730c49768a0f4f4
6,Switzerland,Waste and wastewater management,Installations for the incineration of non-haza...,CH.CAED/000000227.Facility,TRIDEL Lausanne,Lausanne,AIR,Carbon dioxide (CO2),2013,6,...,EUROPE,12.605711,16.682035,18.086070,10.801379,13.061859,15.265277,1,Laura Meza MD,2f4d156f29b46c9add9d9337453822fc
7,Sweden,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,SE.CAED/10020071.Facility,"SUEZ Recycling AB, Rödjorna, Härlingstorp 4:1 ...",SKARA,AIR,Methane (CH4),2011,2,...,EUROPE,13.973565,15.851127,17.580519,8.870747,9.089316,10.427751,2,Darryl Shaw,29273739379051e4f9bf9e0acf7e44bb
8,Luxembourg,Production and processing of metals,Installations for the processing of ferrous me...,LU.CAED/000029000.FACILITY,ArcelorMittal Dudelange,Dudelange,AIR,Nitrogen oxides (NOX),2010,11,...,EUROPE,16.068441,17.179704,22.614642,4.288972,5.584760,9.199652,0,Gary Sanders,da590dcc8795f77b650876ad6d39dc58
9,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,ES.CAED/004958000.FACILITY,COMPLEXO MEDIOAMBIENTAL DE TRATAMENTO DE RSU E...,SERVIA,AIR,Methane (CH4),2012,2,...,EUROPE,18.081017,21.514301,26.428684,6.271424,8.142678,11.357042,2,Trevor Smith,6d63b2d71bf5749713d69d8b18800669


In [4]:
#Ahora cargamos los ficheros JSON

df_json1 = pd.DataFrame(requests.get('http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first').json(),columns=data1.columns)
df_json2 = pd.DataFrame(requests.get('http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second').json(),columns=data1.columns)
df_json3 = pd.DataFrame(requests.get('http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third').json(),columns=data1.columns)

In [5]:
#Unimos todos los trains

frames = [data1, data2, df_json1, df_json2, df_json3]
result = pd.concat(frames)
df1 = pd.DataFrame(result)
df1 = df1.dropna(subset=['pollutant'])
df1['pollutant'] = df1['pollutant'].map({'Nitrogen oxides (NOX)':'0','Carbon dioxide (CO2)':'1','Methane (CH4)':'2'}).astype(int)

In [6]:
#Ahora trabajaremos con el test
data = pd.read_csv('test_x.csv', sep = ',',on_bad_lines='skip')
data

,test_index,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,0,Poland,3,Mineral industry,3(a),Underground mining and related operations,PL.MŚ/000002357.FACILITY,Polska Grupa Górnicza sp. z o.o. Oddział KWK R...,Rydułtowy,AIR,...,EUROPE,14.080054,14.855940,18.474836,10.278561,11.381181,13.480752,1,Brittany Buck,826b1de9dad293ae3e4f9cbaf6cf3420
1,1,Luxembourg,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,LU.CAED/000012000.FACILITY,Sidec,Diekirch,AIR,...,EUROPE,16.052425,17.623877,22.622900,6.625910,8.840137,13.422924,0,Lauren Fisher,ed30a6667b40ba0a66198b3173e7353f
2,2,Netherlands,1,Energy sector,1(c),Thermal power stations and other combustion in...,NL.EEA/212857.FACILITY,Nuon Power Generation BV (Eemshaven),Eemshaven,AIR,...,EUROPE,13.647318,15.541979,17.818521,5.669436,8.403322,11.276388,2,Linda Thompson,78e1082c3cfef3bdf3554da8d6afcc34
3,3,Sweden,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,SE.CAED/10013901.Facility,HÖGBYTORPS AVFALLSANLÄGGNING,BRO,AIR,...,EUROPE,16.336762,17.458113,19.961777,6.160560,7.571596,9.443572,2,Bethany Mcmillan,27f959641950d381869d746d7d0e7d4e
4,4,Portugal,1,Energy sector,1(c),Thermal power stations and other combustion in...,PT.EEA/133926.FACILITY,SPCG - Sociedade Portuguesa de Co-Geração Eléc...,SETÚBAL,AIR,...,EUROPE,21.516602,20.532473,21.617137,10.964012,11.548033,12.624114,2,Sarah Hoffman,1cb71655d9e0bd5cedb2320bf5fdd8f7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24475,24475,Austria,1,Energy sector,1(c),Thermal power stations and other combustion in...,AT.CAED/9008390486467.FACILITY,Trans Austria Gasleitung GmbH,Lechen,AIR,...,EUROPE,11.243493,16.527223,18.114884,3.194254,3.901992,6.437241,2,Crystal Ramirez,a4c235acc3cd402a365de2ff369aa5b1
24476,24476,Spain,1,Energy sector,1(c),Thermal power stations and other combustion in...,ES.CAED/003589000.FACILITY,CENTRAL TÉRMICA DE ESCOMBRERAS,CARTAGENA,AIR,...,EUROPE,14.645536,17.044528,24.602476,9.358224,8.708120,13.258552,0,Amber Grant,d43a0adde7c3a944e7ed740a2114e4a9
24477,24477,Finland,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,http://paikkatiedot.fi/so/1002031/pf/Productio...,"Etelä-Karjalan Jätehuolto Oy, Jätekeskus",Konnunsuo,AIR,...,EUROPE,16.789255,18.595172,24.997173,12.047347,14.686131,17.699271,2,Tiffany Smith,1b2b2db0d413ec0745c8c108317b9529
24478,24478,Netherlands,4,Chemical industry,4(b),Chemical installations for the production on a...,NL.RIVM/000104710.FACILITY,ESD-SIC BV,Farmsum,AIR,...,EUROPE,14.899569,17.112356,19.236777,14.977559,17.846682,20.892999,1,Martin Owens,fe708d5280526c56317bf78b26a1dbd3


In [7]:
df_test = pd.DataFrame(data, columns=df1.columns)
df_test = df_test.drop(['pollutant'],axis=1) 
df_test

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,reportingYear,MONTH,DAY,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,Poland,Mineral industry,Underground mining and related operations,PL.MŚ/000002357.FACILITY,Polska Grupa Górnicza sp. z o.o. Oddział KWK R...,Rydułtowy,AIR,2017,8,16,EUROPE,14.080054,14.855940,18.474836,10.278561,11.381181,13.480752,1,Brittany Buck,826b1de9dad293ae3e4f9cbaf6cf3420
1,Luxembourg,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,LU.CAED/000012000.FACILITY,Sidec,Diekirch,AIR,2008,11,22,EUROPE,16.052425,17.623877,22.622900,6.625910,8.840137,13.422924,0,Lauren Fisher,ed30a6667b40ba0a66198b3173e7353f
2,Netherlands,Energy sector,Thermal power stations and other combustion in...,NL.EEA/212857.FACILITY,Nuon Power Generation BV (Eemshaven),Eemshaven,AIR,2014,9,19,EUROPE,13.647318,15.541979,17.818521,5.669436,8.403322,11.276388,2,Linda Thompson,78e1082c3cfef3bdf3554da8d6afcc34
3,Sweden,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,SE.CAED/10013901.Facility,HÖGBYTORPS AVFALLSANLÄGGNING,BRO,AIR,2011,7,17,EUROPE,16.336762,17.458113,19.961777,6.160560,7.571596,9.443572,2,Bethany Mcmillan,27f959641950d381869d746d7d0e7d4e
4,Portugal,Energy sector,Thermal power stations and other combustion in...,PT.EEA/133926.FACILITY,SPCG - Sociedade Portuguesa de Co-Geração Eléc...,SETÚBAL,AIR,2010,6,23,EUROPE,21.516602,20.532473,21.617137,10.964012,11.548033,12.624114,2,Sarah Hoffman,1cb71655d9e0bd5cedb2320bf5fdd8f7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24475,Austria,Energy sector,Thermal power stations and other combustion in...,AT.CAED/9008390486467.FACILITY,Trans Austria Gasleitung GmbH,Lechen,AIR,2014,11,16,EUROPE,11.243493,16.527223,18.114884,3.194254,3.901992,6.437241,2,Crystal Ramirez,a4c235acc3cd402a365de2ff369aa5b1
24476,Spain,Energy sector,Thermal power stations and other combustion in...,ES.CAED/003589000.FACILITY,CENTRAL TÉRMICA DE ESCOMBRERAS,CARTAGENA,AIR,2007,12,13,EUROPE,14.645536,17.044528,24.602476,9.358224,8.708120,13.258552,0,Amber Grant,d43a0adde7c3a944e7ed740a2114e4a9
24477,Finland,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,http://paikkatiedot.fi/so/1002031/pf/Productio...,"Etelä-Karjalan Jätehuolto Oy, Jätekeskus",Konnunsuo,AIR,2019,12,12,EUROPE,16.789255,18.595172,24.997173,12.047347,14.686131,17.699271,2,Tiffany Smith,1b2b2db0d413ec0745c8c108317b9529
24478,Netherlands,Chemical industry,Chemical installations for the production on a...,NL.RIVM/000104710.FACILITY,ESD-SIC BV,Farmsum,AIR,2013,3,24,EUROPE,14.899569,17.112356,19.236777,14.977559,17.846682,20.892999,1,Martin Owens,fe708d5280526c56317bf78b26a1dbd3


In [8]:

#Transformamos las columnas de strings relevantes para poder hacer la correlación con la variable polución

df1['countryName']=df1['countryName'].astype('category').cat.codes
df1['eprtrSectorName']=df1['eprtrSectorName'].astype('category').cat.codes
df1['EPRTRAnnexIMainActivityLabel']=df1['EPRTRAnnexIMainActivityLabel'].astype('category').cat.codes
df1['FacilityInspireID']=df1['FacilityInspireID'].astype('category').cat.codes
df1['facilityName']=df1['facilityName'].astype('category').cat.codes
df1['City']=df1['City'].astype('category').cat.codes
df1['targetRelease']=df1['targetRelease'].astype('category').cat.codes
df1['CONTINENT']=df1['CONTINENT'].astype('category').cat.codes
df1['REPORTER NAME']=df1['REPORTER NAME'].astype('category').cat.codes
df1['CITY ID']=df1['CITY ID'].astype('category').cat.codes
df1['max_wind_speed']=df1['max_wind_speed'].astype('category').cat.codes
df1['avg_wind_speed']=df1['avg_wind_speed'].astype('category').cat.codes
df1['avg_wind_speed']=df1['avg_wind_speed'].astype('category').cat.codes
df1['min_wind_speed']=df1['min_wind_speed'].astype('category').cat.codes
df1['max_temp']=df1['max_temp'].astype('category').cat.codes
df1['avg_temp']=df1['avg_temp'].astype('category').cat.codes
df1['DAY WITH FOGS']=df1['DAY WITH FOGS'].astype('category').cat.codes

In [9]:
# Correlación para ver qué variables son últiles

#Si vemos los valores obtenidos en la correlación 'targetRelease' y 'CONTINENT' son NaN porque no varían.


df1_corr = df1.corr()
df1_corr['pollutant']

countryName                     0.106054
eprtrSectorName                 0.382519
EPRTRAnnexIMainActivityLabel    0.117085
FacilityInspireID               0.009342
facilityName                   -0.005832
City                           -0.027610
targetRelease                        NaN
pollutant                       1.000000
CONTINENT                            NaN
max_wind_speed                  0.000051
avg_wind_speed                 -0.002556
min_wind_speed                 -0.003450
max_temp                       -0.003051
avg_temp                       -0.004758
DAY WITH FOGS                   0.020367
REPORTER NAME                  -0.000984
CITY ID                        -0.003006
Name: pollutant, dtype: float64

In [10]:
df1 = df1.drop(['targetRelease'], axis = 1)
df1 = df1.drop(['CONTINENT'], axis = 1)

In [11]:
#dfFinal = pd.DataFrame(df1['countryName'])
#dfFinal['eprtrSectorName'] = df1['eprtrSectorName']
#dfFinal['EPRTRAnnexIMainActivityLabel'] = df1['eprtrSectorName']
#dfFinal

In [20]:
#Hacemos lo mismo con las columnas del test
df_test['countryName']=df_test['countryName'].astype('category').cat.codes
df_test['eprtrSectorName']=df_test['countryName'].astype('category').cat.codes
df_test['EPRTRAnnexIMainActivityLabel']=df_test['countryName'].astype('category').cat.codes
df_test['FacilityInspireID']=df_test['countryName'].astype('category').cat.codes
df_test['facilityName']=df_test['countryName'].astype('category').cat.codes
df_test['City']=df_test['countryName'].astype('category').cat.codes
df_test['targetRelease']=df_test['countryName'].astype('category').cat.codes
df_test['CONTINENT']=df_test['countryName'].astype('category').cat.codes
df_test['REPORTER NAME']=df_test['countryName'].astype('category').cat.codes
df_test['CITY ID']=df_test['countryName'].astype('category').cat.codes
df_test['max_wind_speed']=df_test['max_wind_speed'].astype('category').cat.codes
df_test['avg_wind_speed']=df_test['avg_wind_speed'].astype('category').cat.codes
df_test['avg_wind_speed']=df_test['avg_wind_speed'].astype('category').cat.codes
df_test['min_wind_speed']=df_test['min_wind_speed'].astype('category').cat.codes
df_test['max_temp']=df_test['max_temp'].astype('category').cat.codes
df_test['avg_temp']=df_test['avg_temp'].astype('category').cat.codes
df_test['DAY WITH FOGS']=df_test['DAY WITH FOGS'].astype('category').cat.codes

df_test = df_test.drop(['targetRelease'], axis = 1)
df_test = df_test.drop(['CONTINENT'], axis = 1)

In [14]:
#DecisionTreeRegressor

X = df1.drop(['pollutant'],axis=1) 
Y = df1['pollutant']
DT_model = DecisionTreeRegressor(max_depth=5).fit(X,Y)
DT_predict = DT_model.predict(X).round(decimals = 0) #Predictions on Testing data
print(DT_predict[:])
print(metrics.accuracy_score(df1['pollutant'], DT_predict))

[1. 1. 2. ... 2. 1. 1.]
0.593923325409886


In [15]:
#Linear Regression

X = df1.drop(['pollutant'],axis=1) 
#X = dfFinal
Y = df1['pollutant']
LR_model = LogisticRegression().fit(X, Y)
LR_predict = LR_model.predict(X).round(decimals = 0)
print(LR_predict[:])
print(metrics.accuracy_score(df1['pollutant'], LR_predict))

[0 0 0 ... 0 0 0]
0.39588285487901503


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [16]:
#Kneighbors Classifier
kn = KNeighborsClassifier(n_neighbors = 2)
KN_model = kn.fit(X, Y)
KN_predict = KN_model.predict(X).round(decimals = 0)
print(KN_predict[:])
print(metrics.accuracy_score(df1['pollutant'], KN_predict))

[1 0 2 ... 0 0 1]
0.7527884439568477


In [17]:
#Guardado de datos

In [21]:
X_test = df_test
Y = df1['pollutant']
kn = KNeighborsClassifier(n_neighbors = 2)
KN_model = kn.fit(X, Y)
KN_predict = KN_model.predict(X_test).round(decimals = 0)
data.insert(1, 'Predicted Pollution',KN_predict)

In [22]:
from google.colab import files

data.to_csv('predictions.csv', encoding = 'utf-8-sig') 
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [325]:
data

,test_index,Predicted Pollution,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,0,1.0,Poland,3,Mineral industry,3(a),Underground mining and related operations,PL.MŚ/000002357.FACILITY,Polska Grupa Górnicza sp. z o.o. Oddział KWK R...,Rydułtowy,...,EUROPE,14.080054,14.855940,18.474836,10.278561,11.381181,13.480752,1,Brittany Buck,826b1de9dad293ae3e4f9cbaf6cf3420
1,1,1.0,Luxembourg,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,LU.CAED/000012000.FACILITY,Sidec,Diekirch,...,EUROPE,16.052425,17.623877,22.622900,6.625910,8.840137,13.422924,0,Lauren Fisher,ed30a6667b40ba0a66198b3173e7353f
2,2,1.0,Netherlands,1,Energy sector,1(c),Thermal power stations and other combustion in...,NL.EEA/212857.FACILITY,Nuon Power Generation BV (Eemshaven),Eemshaven,...,EUROPE,13.647318,15.541979,17.818521,5.669436,8.403322,11.276388,2,Linda Thompson,78e1082c3cfef3bdf3554da8d6afcc34
3,3,2.0,Sweden,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,SE.CAED/10013901.Facility,HÖGBYTORPS AVFALLSANLÄGGNING,BRO,...,EUROPE,16.336762,17.458113,19.961777,6.160560,7.571596,9.443572,2,Bethany Mcmillan,27f959641950d381869d746d7d0e7d4e
4,4,1.0,Portugal,1,Energy sector,1(c),Thermal power stations and other combustion in...,PT.EEA/133926.FACILITY,SPCG - Sociedade Portuguesa de Co-Geração Eléc...,SETÚBAL,...,EUROPE,21.516602,20.532473,21.617137,10.964012,11.548033,12.624114,2,Sarah Hoffman,1cb71655d9e0bd5cedb2320bf5fdd8f7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24475,24475,1.0,Austria,1,Energy sector,1(c),Thermal power stations and other combustion in...,AT.CAED/9008390486467.FACILITY,Trans Austria Gasleitung GmbH,Lechen,...,EUROPE,11.243493,16.527223,18.114884,3.194254,3.901992,6.437241,2,Crystal Ramirez,a4c235acc3cd402a365de2ff369aa5b1
24476,24476,1.0,Spain,1,Energy sector,1(c),Thermal power stations and other combustion in...,ES.CAED/003589000.FACILITY,CENTRAL TÉRMICA DE ESCOMBRERAS,CARTAGENA,...,EUROPE,14.645536,17.044528,24.602476,9.358224,8.708120,13.258552,0,Amber Grant,d43a0adde7c3a944e7ed740a2114e4a9
24477,24477,0.0,Finland,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,http://paikkatiedot.fi/so/1002031/pf/Productio...,"Etelä-Karjalan Jätehuolto Oy, Jätekeskus",Konnunsuo,...,EUROPE,16.789255,18.595172,24.997173,12.047347,14.686131,17.699271,2,Tiffany Smith,1b2b2db0d413ec0745c8c108317b9529
24478,24478,0.0,Netherlands,4,Chemical industry,4(b),Chemical installations for the production on a...,NL.RIVM/000104710.FACILITY,ESD-SIC BV,Farmsum,...,EUROPE,14.899569,17.112356,19.236777,14.977559,17.846682,20.892999,1,Martin Owens,fe708d5280526c56317bf78b26a1dbd3
